In [10]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mutual_info_score, roc_curve, auc, confusion_matrix, classification_report, roc_auc_score, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
from mrmr import mrmr_classif
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
from scipy.stats import norm

In [2]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# read data (Baseline)
train_data_Baseline = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\Baseline\6-4ADASYN-1\standardized\train.csv')
test_data_Baseline = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\Baseline\6-4ADASYN-1\standardized\test.csv')

# Selector --- univariate logistic regression
significant_features_Baseline = []
def perform_single_factor_analyses_and_select_features(train_df):
    global significant_features_Baseline  
    significant_features_Baseline = []  
    num_features = len(train_df.columns) - 1  
    for i in range(num_features):
        feature_name = train_df.columns[i+1]
        X = train_df[[feature_name]]
        y = train_df['Label']
        X = sm.add_constant(X)
        model = sm.Logit(y, X).fit()
        print(f"{feature_name} - univariate logistic regression:")
        print(model.summary())
        
        if model.pvalues[1] < 0.05:
            significant_features_Baseline.append(feature_name)
            
    return significant_features_Baseline

selected_features_Baseline = perform_single_factor_analyses_and_select_features(train_data_Baseline)

# Classifier --- multivariate logistic regression
def build_multi_factor_model_Baseline(train_data_Baseline, test_data_Baseline, selected_features_Baseline):
    X_train_Baseline = train_data_Baseline[selected_features_Baseline]
    y_train_Baseline = train_data_Baseline['Label']
    X_test_Baseline = test_data_Baseline[selected_features_Baseline]
    y_test_Baseline = test_data_Baseline['Label']
    
    X_train_sm_Baseline = sm.add_constant(X_train_Baseline)
    X_test_sm_Baseline = sm.add_constant(X_test_Baseline)
    
    model_Baseline = sm.Logit(y_train_Baseline, X_train_sm_Baseline)
    result_Baseline = model_Baseline.fit()
    
    y_pred_probs_train_Baseline = result_Baseline.predict(X_train_sm_Baseline)  
    y_pred_probs_test_Baseline = result_Baseline.predict(X_test_sm_Baseline)  
    y_pred_train_Baseline = (y_pred_probs_train_Baseline > 0.5).astype(int)
    y_pred_test_Baseline = (y_pred_probs_test_Baseline > 0.5).astype(int)
    
    return y_pred_probs_train_Baseline, y_pred_probs_test_Baseline
    
    train_fpr_Baseline, train_tpr_Baseline, _ = roc_curve(y_train_Baseline, y_pred_probs_train_Baseline)
    test_fpr_Baseline, test_tpr_Baseline, _ = roc_curve(y_test_Baseline, y_pred_probs_test_Baseline)
    
    conf_matrix_train_Baseline = confusion_matrix(y_train_Baseline, y_pred_train_Baseline)
    train_tn_Baseline, train_fp_Baseline, _, train_tp_Baseline = conf_matrix_train_Baseline.ravel()
    train_specificity_Baseline = train_tn_Baseline / (train_tn_Baseline + train_fp_Baseline)
    train_ppv_Baseline = train_tp_Baseline / (train_tp_Baseline + _)
    train_npv_Baseline = train_tn_Baseline / (train_tn_Baseline + _)

    conf_matrix_test_Baseline = confusion_matrix(y_test_Baseline, y_pred_test_Baseline)
    test_tn_Baseline, test_fp_Baseline, _, test_tp_Baseline = conf_matrix_test_Baseline.ravel()
    test_specificity_Baseline = test_tn_Baseline / (test_tn_Baseline + test_fp_Baseline)
    test_ppv_Baseline = test_tp_Baseline / (test_tp_Baseline + _)
    test_npv_Baseline = test_tn_Baseline / (test_tn_Baseline + _)

    def calculate_auc_with_ci_Baseline(y_true, y_pred_probs, bootstrap_samples=1000):
        fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
        initial_auc_Baseline = auc(fpr, tpr)
        aucs_Baseline = []
        for _ in range(bootstrap_samples):
            sample_indices = np.random.choice(len(y_pred_probs), len(y_pred_probs), replace=True)
            sampled_probs = y_pred_probs[sample_indices]
            fpr_sample, tpr_sample, _ = roc_curve(y_true[sample_indices], sampled_probs)
            auc_sample = auc(fpr_sample, tpr_sample)
            aucs_Baseline.append(auc_sample)
        auc_mean_Baseline = np.mean(aucs_Baseline)
        auc_low_Baseline, auc_high_Baseline = np.percentile(aucs_Baseline, [2.5, 97.5])
        return auc_mean_Baseline, auc_low_Baseline, auc_high_Baseline
    train_auc_mean_Baseline, train_auc_low_Baseline, train_auc_high_Baseline = calculate_auc_with_ci_Baseline(y_train_Baseline, y_pred_probs_train_Baseline)
    test_auc_mean_Baseline, test_auc_low_Baseline, test_auc_high_Baseline = calculate_auc_with_ci_Baseline(y_test_Baseline, y_pred_probs_test_Baseline)
    
build_multi_factor_model_Baseline(train_data_Baseline, test_data_Baseline, selected_features_Baseline)
y_pred_probs_train_Baseline, y_pred_probs_test_Baseline = build_multi_factor_model_Baseline(train_data_Baseline, test_data_Baseline, selected_features_Baseline)

Optimization terminated successfully.
         Current function value: 0.567424
         Iterations 7
TLR-SUVmax - univariate logistic regression:
                           Logit Regression Results                           
Dep. Variable:                  Label   No. Observations:                  207
Model:                          Logit   Df Residuals:                      205
Method:                           MLE   Df Model:                            1
Date:                Tue, 03 Dec 2024   Pseudo R-squ.:                  0.1790
Time:                        17:25:10   Log-Likelihood:                -117.46
converged:                       True   LL-Null:                       -143.07
Covariance Type:            nonrobust   LLR p-value:                 8.205e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0477      0.172      0.278      0.781      -0.

In [3]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# Read data (CT)
train_data_CT = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_CT_bin50\6-4ADASYN-1\train.csv')
test_data_CT = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_CT_bin50\6-4ADASYN-1\test.csv')

X_train_CT = train_data_CT.iloc[:, 1:]  
y_train_CT = train_data_CT.iloc[:, 0]   
X_test_CT = test_data_CT.iloc[:, 1:]  
y_test_CT = test_data_CT.iloc[:, 0]

# Feature Selector ---- MRMR
try:
    selected_features_CT = mrmr_classif(X=X_train_CT, y=y_train_CT, K=10, n_jobs=1)
    print("Selected features names:", selected_features_CT)

    X_train_selected_top_ten_CT = X_train_CT[selected_features_CT]
    X_test_selected_top_ten_CT = X_test_CT[selected_features_CT]
except Exception as e:
    print("An error occurred:", e)

print("Selected features names:", selected_features_CT)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 101.38it/s]

Selected features names: ['CT_wavelet.LLH_glcm_Idn', 'CT_wavelet.LHL_firstorder_Median', 'CT_original_shape_Elongation', 'CT_wavelet.LHL_glcm_MaximumProbability', 'CT_wavelet.LLL_glcm_Correlation', 'CT_log.sigma.1.mm.3D_firstorder_90Percentile', 'CT_wavelet.HLH_glcm_MCC', 'CT_wavelet.HHL_firstorder_Median', 'CT_wavelet.HHL_ngtdm_Contrast', 'CT_original_firstorder_10Percentile']
Selected features names: ['CT_wavelet.LLH_glcm_Idn', 'CT_wavelet.LHL_firstorder_Median', 'CT_original_shape_Elongation', 'CT_wavelet.LHL_glcm_MaximumProbability', 'CT_wavelet.LLL_glcm_Correlation', 'CT_log.sigma.1.mm.3D_firstorder_90Percentile', 'CT_wavelet.HLH_glcm_MCC', 'CT_wavelet.HHL_firstorder_Median', 'CT_wavelet.HHL_ngtdm_Contrast', 'CT_original_firstorder_10Percentile']


In [4]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# Read data (PET)
### If the data is post-harmonized, no additional feature selection process is required.
train_data_PET = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\post-Combat\mRMR\train.csv')
test_data_PET = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\post-Combat\mRMR\test.csv')

X_train_PET = train_data_PET.iloc[:, 1:]  
y_train_PET = train_data_PET.iloc[:, 0]   
X_test_PET = test_data_PET.iloc[:, 1:]   
y_test_PET = test_data_PET.iloc[:, 0]    

# Feature Selector ---- MRMR
#try:
#    selected_features_PET = mrmr_classif(X=X_train_PET, y=y_train_PET, K=10, n_jobs=1)
#    print("Selected features names:", selected_features_PET)

#    X_train_selected_top_ten_PET = X_train_PET[selected_features_PET]
#    X_test_selected_top_ten_PET = X_test_PET[selected_features_PET]
#except Exception as e:
#    print("An error occurred:", e)

#print("Selected features names:", selected_features_PET)

In [5]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
#Classifier (CT) --- Random Forest
np.random.seed(0)
param_grid_CT = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20, 30], 
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],    
    'bootstrap': [True, False]          
}
rf = RandomForestClassifier()
grid_search_CT = GridSearchCV(estimator=rf, param_grid=param_grid_CT, cv=5, scoring='accuracy')
grid_search_CT.fit(X_train_selected_top_ten_CT, y_train_CT)
print("Best parameters:", grid_search_CT.best_params_)
best_model_CT = grid_search_CT.best_estimator_

y_train_pred_cv_CT = cross_val_predict(best_model_CT, X_train_selected_top_ten_CT, y_train_CT, cv=5)
y_train_pred_proba_cv_CT = cross_val_predict(best_model_CT, X_train_selected_top_ten_CT, y_train_CT, cv=5, method='predict_proba')[:, 1]
train_precision_cv_CT, train_recall_cv_CT, train_f1_cv_CT, _ = precision_recall_fscore_support(y_train_CT, y_train_pred_cv_CT, average='binary')
y_test_pred_CT = best_model_CT.predict(X_test_selected_top_ten_CT)
y_test_pred_proba_CT = best_model_CT.predict_proba(X_test_selected_top_ten_CT)[:, 1]
test_precision_CT, test_recall_CT, test_f1_CT, _ = precision_recall_fscore_support(y_test_CT, y_test_pred_CT, average='binary')

train_accuracy_cv_CT = cross_val_score(best_model_CT, X_train_selected_top_ten_CT, y_train_CT, cv=5, scoring='accuracy').mean()
test_accuracy_CT = accuracy_score(y_test_CT, y_test_pred_CT)
conf_mat_train_cv_CT = confusion_matrix(y_train_CT, y_train_pred_cv_CT)
conf_mat_test_CT = confusion_matrix(y_test_CT, y_test_pred_CT)
sensitivity_train_CT = conf_mat_train_cv_CT[1, 1] / (conf_mat_train_cv_CT[1, 1] + conf_mat_train_cv_CT[1, 0])  # TP / (TP + FN)
sensitivity_test_CT = conf_mat_test_CT[1, 1] / (conf_mat_test_CT[1, 1] + conf_mat_test_CT[1, 0])  # TP / (TP + FN)
specificity_train_CT = conf_mat_train_cv_CT[0, 0] / (conf_mat_train_cv_CT[0, 0] + conf_mat_train_cv_CT[0, 1])
specificity_test_CT = conf_mat_test_CT[0, 0] / (conf_mat_test_CT[0, 0] + conf_mat_test_CT[0, 1])
ppv_train_CT = conf_mat_train_cv_CT[1, 1] / (conf_mat_train_cv_CT[1, 1] + conf_mat_train_cv_CT[0, 1])
npv_train_CT = conf_mat_train_cv_CT[0, 0] / (conf_mat_train_cv_CT[0, 0] + conf_mat_train_cv_CT[1, 0])
ppv_test_CT = conf_mat_test_CT[1, 1] / (conf_mat_test_CT[1, 1] + conf_mat_test_CT[0, 1])
npv_test_CT = conf_mat_test_CT[0, 0] / (conf_mat_test_CT[0, 0] + conf_mat_test_CT[1, 0])
train_precision_CT = precision_score(y_train_CT, y_train_pred_cv_CT)
test_precision_CT = precision_score(y_test_CT, y_test_pred_CT)
train_recall_CT = recall_score(y_train_CT, y_train_pred_cv_CT)
test_recall_CT = recall_score(y_test_CT, y_test_pred_CT)
train_f1_CT = f1_score(y_train_CT, y_train_pred_cv_CT)
test_f1_CT = f1_score(y_test_CT, y_test_pred_CT)
print(f"Sensitivity (Cross-Validation_CT): {sensitivity_train_CT:.4f}")
print(f"Specificity (Cross-Validation_CT): {specificity_train_CT:.4f}")
print(f"PPV (Cross-Validation_CT): {ppv_train_CT:.4f}")
print(f"NPV (Cross-Validation_CT): {npv_train_CT:.4f}")
print(f"Precision (Cross-Validation_CT): {train_precision_CT:.4f}")
print(f"Recall (Cross-Validation_CT): {train_recall_CT:.4f}")
print(f"F1 Score (Cross-Validation_CT): {train_f1_CT:.4f}")
print(f"Train Accuracy (Cross-Validation_CT): {train_accuracy_cv_CT:.4f}")
print(f"Sensitivity (Test_CT): {sensitivity_test_CT:.4f}")
print(f"Specificity (Test_CT): {specificity_test_CT:.4f}")
print(f"PPV (Test_CT): {ppv_test_CT:.4f}")
print(f"NPV (Test_CT): {npv_test_CT:.4f}")
print(f"Precision (Test_CT): {test_precision_CT:.4f}")
print(f"Recall (Test_CT): {test_recall_CT:.4f}")
print(f"F1 Score (Test_CT): {test_f1_CT:.4f}")
print(f"Test Accuracy_CT: {test_accuracy_CT:.4f}")

fpr_train_CT, tpr_train_CT, _ = roc_curve(y_train_CT, y_train_pred_proba_cv_CT)
fpr_test_CT, tpr_test_CT, _ = roc_curve(y_test_CT, y_test_pred_proba_CT)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train_CT = bootstrap_roc_auc(y_train_CT, y_train_pred_proba_cv_CT)
auc_train_mean_CT = np.mean(auc_scores_train_CT)
auc_train_std_CT = np.std(auc_scores_train_CT)
auc_train_lower_CT = norm.ppf(0.025, loc=auc_train_mean_CT, scale=auc_train_std_CT)
auc_train_upper_CT = norm.ppf(0.975, loc=auc_train_mean_CT, scale=auc_train_std_CT)
print(f"Cross-Validation AUC_CT: {auc_train_mean_CT:.4f} (95% CI: {auc_train_lower_CT:.4f}, {auc_train_upper_CT:.4f})")

auc_scores_test_CT = bootstrap_roc_auc(y_test_CT, y_test_pred_proba_CT)
auc_test_mean_CT = np.mean(auc_scores_test_CT)
auc_test_std_CT = np.std(auc_scores_test_CT)
auc_test_lower_CT = norm.ppf(0.025, loc=auc_test_mean_CT, scale=auc_test_std_CT)
auc_test_upper_CT = norm.ppf(0.975, loc=auc_test_mean_CT, scale=auc_test_std_CT)
print(f"Test AUC_CT: {auc_test_mean_CT:.4f} (95% CI: {auc_test_lower_CT:.4f}, {auc_test_upper_CT:.4f})")

Best parameters: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Sensitivity (Cross-Validation_CT): 0.8144
Specificity (Cross-Validation_CT): 0.7818
PPV (Cross-Validation_CT): 0.7670
NPV (Cross-Validation_CT): 0.8269
Precision (Cross-Validation_CT): 0.7670
Recall (Cross-Validation_CT): 0.8144
F1 Score (Cross-Validation_CT): 0.7900
Train Accuracy (Cross-Validation_CT): 0.7978
Sensitivity (Test_CT): 0.6579
Specificity (Test_CT): 0.7917
PPV (Test_CT): 0.6250
NPV (Test_CT): 0.8143
Precision (Test_CT): 0.6250
Recall (Test_CT): 0.6579
F1 Score (Test_CT): 0.6410
Test Accuracy_CT: 0.7455
Cross-Validation AUC_CT: 0.8904 (95% CI: 0.8473, 0.9335)
Test AUC_CT: 0.8141 (95% CI: 0.7296, 0.8987)


In [7]:
### One example of dual-modality PET-CT stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
#Classifier (PET) --- Logistic Regression
np.random.seed(0)
param_grid_PET = {
    'C': [0.1, 1.0, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'max_iter': [1000],
    'tol': [1e-4]
}
logistic = LogisticRegression()
grid_search_PET = GridSearchCV(estimator=logistic, param_grid=param_grid_PET, cv=5, scoring='accuracy')
#grid_search_PET.fit(X_train_selected_top_ten_PET, y_train_PET)
grid_search_PET.fit(X_train_PET, y_train_PET)
print("Best parameters:", grid_search_PET.best_params_)
best_model_PET = grid_search_PET.best_estimator_

#y_train_pred_cv_PET = cross_val_predict(best_model_PET, X_train_selected_top_ten_PET, y_train_PET, cv=5)
y_train_pred_cv_PET = cross_val_predict(best_model_PET, X_train_PET, y_train_PET, cv=5)
#y_train_pred_proba_cv_PET = cross_val_predict(best_model_PET, X_train_selected_top_ten_PET, y_train_PET, cv=5, method='predict_proba')[:, 1]
y_train_pred_proba_cv_PET = cross_val_predict(best_model_PET, X_train_PET, y_train_PET, cv=5, method='predict_proba')[:, 1]
train_precision_cv_PET, train_recall_cv_PET, train_f1_cv_PET, _ = precision_recall_fscore_support(y_train_PET, y_train_pred_cv_PET, average='binary')
#y_test_pred_PET = best_model_PET.predict(X_test_selected_top_ten_PET)
y_test_pred_PET = best_model_PET.predict(X_test_PET)
#y_test_pred_proba_PET = best_model_PET.predict_proba(X_test_selected_top_ten_PET)[:, 1]
y_test_pred_proba_PET = best_model_PET.predict_proba(X_test_PET)[:, 1]
test_precision_PET, test_recall_PET, test_f1_PET, _ = precision_recall_fscore_support(y_test_PET, y_test_pred_PET, average='binary')

#train_accuracy_cv_PET = cross_val_score(best_model_PET, X_train_selected_top_ten_PET, y_train_PET, cv=5, scoring='accuracy').mean()
train_accuracy_cv_PET = cross_val_score(best_model_PET, X_train_PET, y_train_PET, cv=5, scoring='accuracy').mean()
test_accuracy_PET = accuracy_score(y_test_PET, y_test_pred_PET)
conf_mat_train_cv_PET = confusion_matrix(y_train_PET, y_train_pred_cv_PET)
conf_mat_test_PET = confusion_matrix(y_test_PET, y_test_pred_PET)
sensitivity_train_PET = conf_mat_train_cv_PET[1, 1] / (conf_mat_train_cv_PET[1, 1] + conf_mat_train_cv_PET[1, 0])  # TP / (TP + FN)
sensitivity_test_PET = conf_mat_test_PET[1, 1] / (conf_mat_test_PET[1, 1] + conf_mat_test_PET[1, 0])  # TP / (TP + FN)
specificity_train_PET = conf_mat_train_cv_PET[0, 0] / (conf_mat_train_cv_PET[0, 0] + conf_mat_train_cv_PET[0, 1])
specificity_test_PET = conf_mat_test_PET[0, 0] / (conf_mat_test_PET[0, 0] + conf_mat_test_PET[0, 1])
ppv_train_PET = conf_mat_train_cv_PET[1, 1] / (conf_mat_train_cv_PET[1, 1] + conf_mat_train_cv_PET[0, 1])
npv_train_PET = conf_mat_train_cv_PET[0, 0] / (conf_mat_train_cv_PET[0, 0] + conf_mat_train_cv_PET[1, 0])
ppv_test_PET = conf_mat_test_PET[1, 1] / (conf_mat_test_PET[1, 1] + conf_mat_test_PET[0, 1])
npv_test_PET = conf_mat_test_PET[0, 0] / (conf_mat_test_PET[0, 0] + conf_mat_test_PET[1, 0])
train_precision_PET = precision_score(y_train_PET, y_train_pred_cv_PET)
test_precision_PET = precision_score(y_test_PET, y_test_pred_PET)
train_recall_PET = recall_score(y_train_PET, y_train_pred_cv_PET)
test_recall_PET = recall_score(y_test_PET, y_test_pred_PET)
train_f1_PET = f1_score(y_train_PET, y_train_pred_cv_PET)
test_f1_PET = f1_score(y_test_PET, y_test_pred_PET)
print(f"Sensitivity (Cross-Validation_PET): {sensitivity_train_PET:.4f}")
print(f"Specificity (Cross-Validation_PET): {specificity_train_PET:.4f}")
print(f"PPV (Cross-Validation_PET): {ppv_train_PET:.4f}")
print(f"NPV (Cross-Validation_PET): {npv_train_PET:.4f}")
print(f"Precision (Cross-Validation_PET): {train_precision_PET:.4f}")
print(f"Recall (Cross-Validation_PET): {train_recall_PET:.4f}")
print(f"F1 Score (Cross-Validation_PET): {train_f1_PET:.4f}")
print(f"Train Accuracy (Cross-Validation_PET): {train_accuracy_cv_PET:.4f}")
print(f"Sensitivity (Test_PET): {sensitivity_test_PET:.4f}")
print(f"Specificity (Test_PET): {specificity_test_PET:.4f}")
print(f"PPV (Test_PET): {ppv_test_PET:.4f}")
print(f"NPV (Test_PET): {npv_test_PET:.4f}")
print(f"Precision (Test_PET): {test_precision_PET:.4f}")
print(f"Recall (Test_PET): {test_recall_PET:.4f}")
print(f"F1 Score (Test_PET): {test_f1_PET:.4f}")
print(f"Test Accuracy_PET: {test_accuracy_PET:.4f}")

fpr_train_PET, tpr_train_PET, _ = roc_curve(y_train_PET, y_train_pred_proba_cv_PET)
fpr_test_PET, tpr_test_PET, _ = roc_curve(y_test_PET, y_test_pred_proba_PET)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train_PET = bootstrap_roc_auc(y_train_PET, y_train_pred_proba_cv_PET)
auc_train_mean_PET = np.mean(auc_scores_train_PET)
auc_train_std_PET = np.std(auc_scores_train_PET)
auc_train_lower_PET = norm.ppf(0.025, loc=auc_train_mean_PET, scale=auc_train_std_PET)
auc_train_upper_PET = norm.ppf(0.975, loc=auc_train_mean_PET, scale=auc_train_std_PET)
print(f"Cross-Validation AUC_PET: {auc_train_mean_PET:.4f} (95% CI: {auc_train_lower_PET:.4f}, {auc_train_upper_PET:.4f})")

auc_scores_test_PET = bootstrap_roc_auc(y_test_PET, y_test_pred_proba_PET)
auc_test_mean_PET = np.mean(auc_scores_test_PET)
auc_test_std_PET = np.std(auc_scores_test_PET)
auc_test_lower_PET = norm.ppf(0.025, loc=auc_test_mean_PET, scale=auc_test_std_PET)
auc_test_upper_PET = norm.ppf(0.975, loc=auc_test_mean_PET, scale=auc_test_std_PET)
print(f"Test AUC_PET: {auc_test_mean_PET:.4f} (95% CI: {auc_test_lower_PET:.4f}, {auc_test_upper_PET:.4f})")

Best parameters: {'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 0.0001}
Sensitivity (Cross-Validation_PET): 0.6701
Specificity (Cross-Validation_PET): 0.7727
PPV (Cross-Validation_PET): 0.7222
NPV (Cross-Validation_PET): 0.7265
Precision (Cross-Validation_PET): 0.7222
Recall (Cross-Validation_PET): 0.6701
F1 Score (Cross-Validation_PET): 0.6952
Train Accuracy (Cross-Validation_PET): 0.7243
Sensitivity (Test_PET): 0.6579
Specificity (Test_PET): 0.8056
PPV (Test_PET): 0.6410
NPV (Test_PET): 0.8169
Precision (Test_PET): 0.6410
Recall (Test_PET): 0.6579
F1 Score (Test_PET): 0.6494
Test Accuracy_PET: 0.7545
Cross-Validation AUC_PET: 0.7776 (95% CI: 0.7153, 0.8400)
Test AUC_PET: 0.8605 (95% CI: 0.7957, 0.9254)


In [13]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
#Classifier (Meta) --- Logistic Regression/SVM
X_train_meta_features = np.column_stack([
    y_pred_probs_train_Baseline,  
    y_train_pred_proba_cv_CT,      
    y_train_pred_proba_cv_PET    
])

X_test_meta_features = np.column_stack([
    y_pred_probs_test_Baseline,   
    y_test_pred_proba_CT,         
    y_test_pred_proba_PET         
])

np.random.seed(0)
lr = LogisticRegression()
param_grid = {
    'C': [0.1, 1.0, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'max_iter': [1000],
    'tol': [1e-4]
}
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_meta_features, y_train_CT)
best_params = grid_search.best_params_
print("Best parameters for Logistic Regression Meta-learner:", best_params)

best_meta_model = grid_search.best_estimator_
y_train_meta_pred = cross_val_predict(best_meta_model, X_train_meta_features, y_train_CT, cv=5)
y_test_meta_pred = best_meta_model.predict(X_test_meta_features)
y_train_meta_pred_proba = cross_val_predict(best_meta_model, X_train_meta_features, y_train_CT, cv=5, method='predict_proba')[:, 1]
y_test_meta_pred_proba = best_meta_model.predict_proba(X_test_meta_features)[:, 1]

train_accuracy_meta = cross_val_score(best_meta_model, X_train_meta_features, y_train_CT, cv=5, scoring='accuracy').mean()
test_accuracy_meta = accuracy_score(y_test_CT, y_test_meta_pred)
train_precision_meta, train_recall_meta, train_f1_meta, _ = precision_recall_fscore_support(y_train_CT, y_train_meta_pred, average='binary')
test_precision_meta, test_recall_meta, test_f1_meta, _ = precision_recall_fscore_support(y_test_CT, y_test_meta_pred, average='binary')

def calculate_additional_metrics(y_true, y_pred, y_proba):
    conf_matrix = confusion_matrix(y_true, y_pred)
    sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])  # TP / (TP + FN)
    specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])  # TN / (TN + FP)
    ppv = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[0, 1])  # TP / (TP + FP)
    npv = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0])  # TN / (TN + FN)
    return sensitivity, specificity, ppv, npv, conf_matrix

sensitivity_train_meta, train_specificity, train_ppv, train_npv, conf_mat_train_meta = calculate_additional_metrics(y_train_CT, y_train_meta_pred, y_train_meta_pred_proba)
sensitivity_test_meta, test_specificity, test_ppv, test_npv, conf_mat_test_meta = calculate_additional_metrics(y_test_CT, y_test_meta_pred, y_test_meta_pred_proba)

print(f"Sensitivity (Cross-Validation Meta-learner): {sensitivity_train_meta:.4f}")
print(f"Specificity (Cross-Validation Meta-learner): {train_specificity:.4f}")
print(f"PPV (Cross-Validation Meta-learner): {train_ppv:.4f}")
print(f"NPV (Cross-Validation Meta-learner): {train_npv:.4f}")
print(f"Precision (Cross-Validation Meta-learner): {train_precision_meta:.4f}")
print(f"Recall (Cross-Validation Meta-learner): {train_recall_meta:.4f}")
print(f"F1 Score (Cross-Validation Meta-learner): {train_f1_meta:.4f}")
print(f"Train Accuracy (Cross-Validation  Meta-learner): {train_accuracy_meta:.4f}")
print(f"Sensitivity (Test Meta-learner): {sensitivity_test_meta:.4f}")
print(f"Specificity (Test Meta-learner): {test_specificity:.4f}")
print(f"PPV (Test Meta-learner): {test_ppv:.4f}")
print(f"NPV (Test Meta-learner): {test_npv:.4f}")
print(f"Precision (Test Meta-learner): {test_precision_meta:.4f}")
print(f"Recall (Test Meta-learner): {test_recall_meta:.4f}")
print(f"F1 Score (Test Meta-learner): {test_f1_meta:.4f}")
print(f"Test Accuracy (Test Meta-learner): {test_accuracy_meta:.4f}")

np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

fpr_train_meta, tpr_train_meta, _ = roc_curve(y_train_CT, y_train_meta_pred_proba)
fpr_test_meta, tpr_test_meta, _ = roc_curve(y_test_CT, y_test_meta_pred_proba)
auc_scores_train_meta = bootstrap_roc_auc(y_train_CT, y_train_meta_pred_proba)
auc_scores_test_meta = bootstrap_roc_auc(y_test_CT, y_test_meta_pred_proba)
auc_train_mean_meta = np.mean(auc_scores_train_meta)
auc_train_std_meta = np.std(auc_scores_train_meta)
auc_test_mean_meta = np.mean(auc_scores_test_meta)
auc_test_std_meta = np.std(auc_scores_test_meta)
auc_train_lower_meta = norm.ppf(0.025, loc=auc_train_mean_meta, scale=auc_train_std_meta)
auc_train_upper_meta = norm.ppf(0.975, loc=auc_train_mean_meta, scale=auc_train_std_meta)
auc_test_lower_meta = norm.ppf(0.025, loc=auc_test_mean_meta, scale=auc_test_std_meta)
auc_test_upper_meta = norm.ppf(0.975, loc=auc_test_mean_meta, scale=auc_test_std_meta)
print(f"Meta-learner Cross-Validation AUC: {auc_train_mean_meta:.4f} (95% CI: {auc_train_lower_meta:.4f}, {auc_train_upper_meta:.4f})")
print(f"Meta-learner Test AUC: {auc_test_mean_meta:.4f} (95% CI: {auc_test_lower_meta:.4f}, {auc_test_upper_meta:.4f})")

Best parameters for Logistic Regression Meta-learner: {'C': 10, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}
Sensitivity (Cross-Validation Meta-learner): 0.8351
Specificity (Cross-Validation Meta-learner): 0.8818
PPV (Cross-Validation Meta-learner): 0.8617
NPV (Cross-Validation Meta-learner): 0.8584
Precision (Cross-Validation Meta-learner): 0.8617
Recall (Cross-Validation Meta-learner): 0.8351
F1 Score (Cross-Validation Meta-learner): 0.8482
Train Accuracy (Cross-Validation  Meta-learner): 0.8600
Sensitivity (Test Meta-learner): 0.7368
Specificity (Test Meta-learner): 0.8750
PPV (Test Meta-learner): 0.7568
NPV (Test Meta-learner): 0.8630
Precision (Test Meta-learner): 0.7568
Recall (Test Meta-learner): 0.7368
F1 Score (Test Meta-learner): 0.7467
Test Accuracy (Test Meta-learner): 0.8273
Meta-learner Cross-Validation AUC: 0.9191 (95% CI: 0.8816, 0.9566)
Meta-learner Test AUC: 0.8754 (95% CI: 0.8013, 0.9495)
